In [106]:
from copy import copy

In [2]:
filename = 'lexicon.txt'
lexicon_file = open(filename, 'r')
lexicon = [line.rstrip() for line in lexicon_file.readlines()]

In [111]:
class TrieNode:
    def __init__(self, eow: bool = False):
        self.eow = eow  # end of word
        self.children = {}

    def add(self, char, final_char: bool = False):
        assert char not in self.children.keys()
        self.children[char] = TrieNode(final_char)

    def __repr__(self):
        return '{} {}'.format('eow' if self.eow else '', self.children, )


class Trie:
    def __init__(self):
        self.root = TrieNode()
        self.search_candidate = ''

    def add_word(self, word: str):
        if word in self:
            return

        word = list(word)
        node = self.root
        
        # get to the most recent node
        while word[0] in node.children.keys():
            node = node.children[word.pop(0)]
            
        # add more nodes
        while word:
            char = word.pop(0)
            node.add(char, True if not word else False)
            node = node.children[char]

    def add_words(self, words: list):
        for word in words:
            self.add_word(word)
            
    def find_words(self, letters: list):
        valid_words = set()
        search_candidate = []
        letters_remaining = letters

        def search(v):            
            if v.eow and ''.join(search_candidate) not in valid_words:
                valid_words.add(''.join(search_candidate))
            
            for l in copy(letters_remaining):
                if l in v.children:
                    search_candidate.append(l)
                    letters_remaining.remove(l)
                    search(v.children[l])
                    del search_candidate[-1]
                    letters_remaining.append(l)

        search(self.root)
        return valid_words

    def __contains__(self, word: str):
        word = list(word)
        node = self.root
        while word:
            char = word.pop(0)
            try:
                node = node.children[char]
            except KeyError:
                return False
        return True if node.eow else False

    def __repr__(self):
        return str(self.root)

In [112]:
trie = Trie()
trie.add_words(lexicon[:40000])

In [113]:
letters = ['A', 'B', 'C']
words = trie.find_words(letters)

searching . remaining ['A', 'B', 'C']
searching A. remaining ['B', 'C']
searching AB. remaining ['C']
searching ABC. remaining []
searching AC. remaining ['B']
searching B. remaining ['C', 'A']
searching BA. remaining ['C']
searching BAC. remaining []
searching C. remaining ['A', 'B']
searching CA. remaining ['B']
searching CAB. remaining []


In [110]:
words

{'AB', 'BA', 'BAC', 'CAB'}

In [86]:
'C' in trie.root.children

True

In [72]:
words

{'AB', 'BA', 'BAC'}

In [ ]:
def dfs(n, edges):
    """Given a list of all edges of a graph with vertices 0 to n-1,
    will explore every vertex, depth first, without recursion."""

    undiscovered = set(range(n))

    first_vertex = root
    undiscovered.add(first_vertex)
    stack = [first_vertex]              # the stack is a list of vertices to explore
    while stack:
        v = stack.pop()
        if v in undiscovered:
            print('Just discovered vertex {}.'.format(v))
            undiscovered.remove(v)
            for w in edges_by_vertex[v]:
                stack.append(w)


dfs(8, [[0, 1], [0, 2], [0, 4], [1, 3], [1, 5], [2, 6], [4, 5], ])

In [ ]:
    def find_words(self, letters: list):
        """Given a list of all edges of a graph with vertices 0 to n-1,
        will explore every vertex, depth first, without recursion."""
        valid_words = []
        
        first_vertex = self.root
        discovered = {first_vertex}
        stack = [first_vertex]              # the stack is a list of vertices to explore
        c = []                              # the current path
        while stack:
            v = stack.pop()
            if v not in discovered:
                discovered.add(v)
                if v.eow:
                    valid_words.append(''.join(c))
                for w in edges_by_vertex[v]:
                    stack.append(w)